# Chat Models大语言模型

## 简介

在Agent智能体中，大语言模型提供理解、决策、调度等能力，是最基础的大脑模块。因此，本文我们介绍如何上手使用大语言模型。

简而言之，大语言模型基于深度学习技术，能够自动理解和生成人类自然语言文本的模型，广泛应用于文本生成、机器翻译、自动摘要、问答系统、情感分析等任务。

目前市面上已经有很多大语言模型，比如OpenAI的ChatGPT、百度的文心一言等等。大语言模型通常具有大量参数，对运行机器的显存和算量有很高的要求。为了方便大家使用，这些大语言模型都有对外提供相应的调用接口。

在`ERNIE Bot Agent`中，我们支持快速调用文心一言的多个模型，包括`ernie-3.5`、`ernie-turbo`、`ernie-4.0`和`ernie-longtext`。

| 模型名称 | 说明 | 功能 | 输入token数量上限 |
|:--- | :--- | :--- | :--- |
| ernie-3.5 | 文心大模型3.5版本。具备优秀的知识增强和内容生成能力，在文本创作、问答、推理和代码生成等方面表现出色。 | 对话补全，函数调用 | 3000 |
| ernie-turbo | 文心大模型。相比ernie-3.5模型具备更快的响应速度和学习能力，API调用成本更低。 | 对话补全 |  3000 |
| ernie-4.0 | 文心大模型4.0版本，具备目前系列模型中最优的理解和生成能力。 | 对话补全，函数调用 |  3000 |
| ernie-longtext | 文心大模型。在ernie-3.5模型的基础上增强了对长对话上下文的支持，输入token数量上限为7000。 | 对话补全，函数调用 |  7000 |

## 上手使用文心一言

### 安装

大家参考如下，灵活选用快速安装或者源码安装。

1）源码安装

执行如下命令，使用源码安装ERNIE Bot Agent（要求Python >= 3.8)。



In [ ]:
!git clone https://github.com/PaddlePaddle/ERNIE-Bot-SDK.git
%cd ~/ERNIE-Bot-SDK

# 切换到特定分支，默认是develop分支
# !git checkout -b xxx origin/xxx

# 首先安装Ernie Bot SDK
!pip install ./erniebot/

# 然后安装ERNIE Bot Agent
!pip install ./erniebot-agent 

2）快速安装（暂不支持）

执行如下命令，快速安装最新版本ERNIE Bot Agent（要求Python >= 3.8)。

In [ ]:
# 安装核心模块
!pip install --upgrade erniebot-agent

### 鉴权

大家在使用ERNIE Bot Agent之前，需要完成鉴权步骤：

* 在[AI Studio星河社区](https://aistudio.baidu.com/index)注册并登录账号
* 在个人中心的[访问令牌页面](https://aistudio.baidu.com/index/accessToken)获取用户凭证`Access Token`
* 通过环境变量或者`Python`代码设置`Access Token`

In [ ]:
%env EB_AGENT_ACCESS_TOKEN=xxxxxxx

### 使用文心一言

首先，导入必要的依赖库。

In [ ]:
import json
import asyncio
from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import HumanMessage, AIMessage, SystemMessage, FunctionMessage

出于使用场景和性能的考虑，`ERNIE Bot Agent`只提供异步接口来调用文心一言模型。关于异步接口的详细介绍，请参考[asyncio文档](https://docs.python.org/3/library/asyncio.html)。

如下示例，我们首先创建文心一言`ernie-3.5`模型，然后两次调用`chat`接口传入只有单条`HumanMessage`的数组，文心一言模型会对单条`HumanMessage`做出回答，返回一条`AIMessage`。

In [ ]:
async def demo_1():
    model = ERNIEBot(model="ernie-3.5")                         # 创建模型
    human_message = HumanMessage(content='你好，你是谁')          # 定义输入信息
    ai_message = await model.chat(messages=[human_message])     # 调用模型chat接口，非流式返回
    print(ai_message.content, '\n')                             # 输出结果

    human_message = HumanMessage(content='推荐三个深圳有名的景点')  # 定义输入信息
    ai_message = await model.chat(messages=[human_message],     # 调用模型chat接口，流式返回
                                        stream=True)
    async for chunk in ai_message:                              # 流式输出结果
        print(chunk.content, end='')

await demo_1()

如果希望文心一言模型能够根据多轮对话的上下文进行回答，我们需要将前面对话的输入输出`Message`带入后面对话，具体参考如下代码。

In [ ]:
async def demo_2():
    model = ERNIEBot(model="ernie-3.5")
    messages = []                       # 使用列表保存所有Message信息

    messages.append(HumanMessage(content='推荐三个深圳有名的景点'))
    ai_message = await model.chat(messages=messages)
    messages.append(ai_message)
    print(ai_message.content, '\n')

    messages.append(HumanMessage(content='根据你推荐的景点，帮我做一份一日游的攻略'))
    ai_message = await model.chat(messages=messages)
    messages.append(ai_message)
    print(ai_message.content, '\n')

await demo_2()

下面示例，我们实现了一个简易的命令行聊天应用，可以和大语言模型网页端一样进行无限畅聊。

In [ ]:
async def demo_3():
    model = ERNIEBot(model='ernie-3.5')
    messages = []

    print('你好，有什么我可以帮助你的吗？ (输入q可以退出聊天)')
    while True:
        prompt = input()
        if prompt == 'q':
            break

        messages.append(HumanMessage(prompt))
        ai_message = await model.chat(messages=messages, stream=True)

        result = ''
        async for chunk in ai_message:
            result += chunk.content
            print(chunk.content, end='')
        print('')
        messages.append(AIMessage(result))

await demo_3()

下面示例，我们展示了文心一言模型如何使用FunctionCall。

In [ ]:
# 定义function函数
def get_student_score(name: str) -> dict:
    info = {'小明': 80, '小红': 90, '小天': 95}
    if name in info:
        return {"score": info[name]}
    else:
        return f"we do not know the score of {name}"

# 定义function描述
functions = [
    {
        "name": "get_student_score",
        "description": "查询学生的分数",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "学生姓名",
                },
            },
            "required": [
                "name",
            ],
        },
        "responses": {
            "type": "object",
            "properties": {
                "score": {
                    "type": "integer",
                    "description": "分数",
                },
            },
        },
    },
]

async def demo():
    model = ERNIEBot(model="ernie-3.5")
    messages = []

    messages.append(HumanMessage("请问小明的分数是多少"))
    ai_message = await model.chat(messages=messages, functions=functions)   # 发送问题，带上function描述
    messages.append(ai_message)

    function_call = ai_message.function_call
    if function_call is not None:                       # 如果返回的AIMessage有触发function，会有function_call字段
        name = function_call['name']                    # function_call的函数名称
        arguments = eval(function_call['arguments'])    # function_call的函数输入实参
        result = eval(name)(**arguments)                # 使用函数实参，调用并执行函数，拿到结果
        print('function_call:', function_call)
        print('function result:', result)

        function_message = FunctionMessage(name=name, content=json.dumps(result, ensure_ascii=False))   # 构建FunctionMessage，封装函数的结果
        messages.append(function_message)

        ai_message = await model.chat(messages=messages, functions=functions)    # 将函数结果返回给模型，进行润色，得到最终输出
        print('final result:', ai_message.content)

await demo()